In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
csv_path = 'D:\\Semester 7\\FYP\\preprocessing\\output_labels.csv'
df = pd.read_csv(csv_path)

# Extract file paths and class labels
file_paths = df['Path'].values
class_labels = df['Class'].values

# Load ECG data from file paths
ecg_data = []
for path in file_paths:
    # Load ECG data from CSV file
    ecg_df = pd.read_csv(path)
    # Assuming your ECG data is in columns I, II, III, AVR, AVL, AVF, V1, V2, V3, V4, V5, V6
    ecg_values = ecg_df[['I', 'II', 'III', 'AVR', 'AVL',
                         'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']].values
    ecg_data.append(ecg_values)

X = np.array(ecg_data)
y = np.array(class_labels)

In [3]:
print(X.shape, y.shape)

(5000, 5000, 12) (5000,)


In [4]:
y

array(['NORM', 'NORM', 'NORM', ..., 'HYP', 'HYP', 'HYP'], dtype=object)

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
# shuufle the data
from sklearn.utils import shuffle
X, y = shuffle(X, y)


In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# No need to reshape X for LSTM input

# Instantiate the encoder
le = LabelEncoder()

# Fit the encoder and transform the labels
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [8]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Dropout , Dense
from tensorflow.keras.optimizers import Adam

In [9]:

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.6))  # Add dropout layer for regularization
model.add(Dense(len(np.unique(y_train)), activation='softmax'))

# Compile the model with a lower learning rate and Adam optimizer
optimizer = Adam(learning_rate=0.0001)  # Specify the learning rate here
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

# Train the model with increased epochs and batch size
model.fit(X_train, y_train, epochs=5, batch_size=32,
          validation_data=(X_test, y_test))

# Evaluate the model and print test accuracy
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Test Accuracy: {accuracy}")

c:\Program Files\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 1997s 16s/step - accuracy: 0.2099 - loss: 1.6081 - val_accuracy: 0.2250 - val_loss: 1.6071
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 2101s 17s/step - accuracy: 0.2419 - loss: 1.6059 - val_accuracy: 0.2350 - val_loss: 1.6051
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 1026s 8s/step - accuracy: 0.2410 - loss: 1.6039 - val_accuracy: 0.2330 - val_loss: 1.6028
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 1725s 14s/step - accuracy: 0.2540 - loss: 1.6008 - val_accuracy: 0.2290 - val_loss: 1.6000
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 1564s 12s/step - accuracy: 0.2642 - loss: 1.5940 - val_accuracy: 0.2360 - val_loss: 1.5959
32/32 ━━━━━━━━━━━━━━━━━━━━ 76s 2s/step - accuracy: 0.2213 - loss: 1.5972
Test Accuracy: 0.23600000143051147
